## CNN
Convolution Neural Networks은 이미지 분야를 다루기에 최적화된 인공 신경망 구조이다.  
크게 Convolution Layer와 Pooling Layer로서 구성되어 있다.  
- Convolution Layer: Convolution 연산을 통해서 이미지의 특징을 추출해내는 역활
- Pooling Layer: 차원 축소하는 연산을 수행한다. **연산량을 감소** 시킬 수 있고, 이미지의 **가장 강한 특징만을 추출**하는 특징 선별효과가 있다.


**Tensorflow API**  
tf.nn.conv2d(
input, 
filter,
strides,
padding,
dilations=[1,1,1,1],
name=None
)

- input: Input Data
- filter: 컨볼루션 연산에 적용할 필터이며 [filter_height, filter_width, in_channels, out_channels]형태의 4-D Tensor만 가능
- strides: 몇 Pixel씩 넘어갈지 지정
- padding: 
 - 'Same': zero padding을 통하여 input과 같은 크기의 이미지가 return
 - 'VALID': 컨볼루션 연산공식에 의해 계산된 가로, 세로, 차원이 리턴
- dilation: Dilation Factor

tf.nn.max_pool(
value, 
ksize,
strides,
name=None
)

- value: Max Pooling을 적용시킬 Input Data
- ksize: Max Pooling 연산에 적용할 필터이며 [batch_filter,height_filter, width_filter, channel_filter]형태의 4-D Tensor만 가능
- strides: 몇 Pixel씩 넘어갈지 지정

tf.nn.avg_pool;(
value, 
ksize,
strides,
name=None
)

- value: Mean Pooling을 적용시킬 Input Data
- ksize: Max Pooling 연산에 적용할 필터이며 [batch_filter,height_filter, width_filter, channel_filter]형태의 4-D Tensor만 가능
- strides: 몇 Pixel씩 넘어갈지 지정

## CNN 실제 구현

텐서플로 라이브러리를 임포트

In [1]:
import tensorflow as tf

MNIST 데이터를 다운받고 불러오는 과정

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


CNN 모델을 정의  
- Convolution Layer: 2개
- Pooling Layer: 2개

x_image를 [-1,28,28,1]을 통하여 Flatten한 1차원의 행렬로서 만든다.  
따라서 W_conv1 이 [5,5,1,32] 즉 5x5 Filter를 Convolution으로 적용시키는데  
Input의 차원은 1차원  
Output의 차원은 32개로 나타낸다는 의미이다.  

In [3]:
#CNN Model Definition
def build_CNN_Classifier(x):
    #MNIST 데이터를 3차원 형태로 reshape(흑백이므로 채널은 1)
    x_image = tf.reshape(x,[-1,28,28,1])
    
    # 1_Convolution_Layer
    # 5 x 5 Filter 32개 적용
    # 28 x 28 => 28 x 28 x 32
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,1,32],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1],padding='SAME')+b_conv1)
    
    # 1_Pooling Layer
    # Max_Pooling => Image size 1/2
    # 28 x 28 x 32 => 14 x 14 x 32
    h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    # 2_Convolution_Layer
    # 5 x 5 Filter 64개 적용
    # 14 x 14 x 32 => 14 x 14 x 64
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,32,64],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)
    
    # 2_Pooling Layer
    # Max_Pooling => Image size 1/2
    # 14 x 14 x 64 => 7 x 7 x 64
    h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    # 완전 연결층
    # 7 x 7 크기를 가진 64개의 activation map을 1024개의 특징들로 변환
    # 7 x 7 x 64(3136) => 1024
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[7*7*64, 1024],stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1,shape=[1024]))
    h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
    
    # 출력층
    # 1024개의 특징들(feature)을 10개의 class로 변환
    # 1024 -> 10
    W_output = tf.Variable(tf.truncated_normal(shape=[1024,10],stddev=5e-2))
    b_output = tf.Variable(tf.constant(0.1,shape=[10]))
    logits = tf.matmul(h_fc1,W_output)+b_output
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

Input 과 Output 데이터를 받을 Place Holder 정의

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

Model 생성

In [5]:
y_pred, logits = build_CNN_Classifier(x)

Instructions for updating:
Colocations handled automatically by placer.


- LosttFunction: Cross_Entropy_with_Softmax
- Optimzer: Adam

In [6]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.AdamOptimizer(1e-04).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



정확도를 출력하기 위한 연산들을 정의

In [7]:
correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Session을 열고 그래프 실행하여 학습 실행

In [8]:
with tf.Session() as sess:
    #모든 변수 초기화
    sess.run(tf.global_variables_initializer())
    
    for i in range(3000):
        #50개씩 MNIST 데이터를 불러온다.
        batch = mnist.train.next_batch(50)
        
        if i % 100 ==0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y:batch[1]})
            print('반복(Epoch): %d, 정확도: %f'%(i, train_accuracy))
            
        sess.run([train_step], feed_dict={x:batch[0], y: batch[1]})
        
    #정확도 측정
    print('Model 정확도: %f'%accuracy.eval(feed_dict={x:mnist.test.images, y: mnist.test.labels}))

반복(Epoch): 0, 정확도: 0.120000
반복(Epoch): 100, 정확도: 0.800000
반복(Epoch): 200, 정확도: 0.920000
반복(Epoch): 300, 정확도: 0.960000
반복(Epoch): 400, 정확도: 0.940000
반복(Epoch): 500, 정확도: 0.940000
반복(Epoch): 600, 정확도: 0.900000
반복(Epoch): 700, 정확도: 0.940000


KeyboardInterrupt: 

accuracy.eval() 에서 .eval()은 run()과 같이 Session을 연 뒤 연산을 실행하는 과정이라고 생각하면 된다.

## CIFAR - 10
CIFAR -10은 총 10개의 레이블로 구성된 이미지 분류를 위한 Dataset이다.  
Image는 32 x 32의 크기의 이미지로 되어있고 또한  
MNIST와 달리 Color Image인 것이 특징이다.

## Dropout
Dropout은 Overfitting을 방지하기 위한 Regularization 기법의 일종이다.  
**Tensorflow API**  
tf.nn.dropout(
x, 
keep_prob,
name=None
)

- x: Dropout을 적용할 Input Data
- keep_prob: 드롭하지 않고 유지할 노드의 비율을 나타내는 scalar 텐서

## CNN을 이용한 CIFAR-10 이미지 분류기 구현

numpy 라이브러리 import

CIFAR-10 데이터셋을 다운받고, 불러오는 과정을 지원하는 helper 함수를 keras 모듈에서 load_data라는 모듈 함수로 제공

In [9]:
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data

데이터를 배치 개수만큼 끊어서 읽어올 수 있는 next_batch 함수 선언

In [10]:
def next_batch(num, data, labels):
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

CNN Model을 정의  
- Convolution Layer: 2개
- Pooling Layer: 2개

In [11]:
#CNN Model Definition
def build_CNN_Classifier(x):
    #MNIST 데이터를 3차원 형태로 reshape(흑백이므로 채널은 1)
    x_image = x
    
    # 1_Convolution_Layer
    # RGB(Color) image를 64개의 feature으로 mapping 과정
    W_conv1 = tf.Variable(tf.truncated_normal(shape=[5,5,3,64],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv1 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1],padding='SAME')+b_conv1)
    
    # 1_Pooling Layer
    h_pool1 = tf.nn.max_pool(h_conv1,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
    
    # 2_Convolution_Layer
    # 64개의 feature을 다시 64개의 feature으로 mapping 하는 과정
    W_conv2 = tf.Variable(tf.truncated_normal(shape=[5,5,64,64],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2,strides=[1,1,1,1],padding='SAME')+b_conv2)
    
    # 2_Pooling Layer
    h_pool2 = tf.nn.max_pool(h_conv2,ksize=[1,3,3,1],strides=[1,2,2,1],padding='SAME')
    
    # 3_Convolution_Layer
    # 64개의 feature을 다시 128개의 feature으로 mapping 하는 과정
    W_conv3 = tf.Variable(tf.truncated_normal(shape=[3,3,64,128],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv3 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv3 = tf.nn.relu(tf.nn.conv2d(h_pool2, W_conv3,strides=[1,1,1,1],padding='SAME')+b_conv3)
    
    
    # 4_Convolution_Layer
    # 128개의 feature을 다시 128개의 feature으로 mapping 하는 과정
    W_conv4 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv4 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv4 = tf.nn.relu(tf.nn.conv2d(h_conv3, W_conv4,strides=[1,1,1,1],padding='SAME')+b_conv4)
    
    
    # 5_Convolution_Layer
    # 128개의 feature을 다시 128개의 feature으로 mapping 하는 과정
    W_conv5 = tf.Variable(tf.truncated_normal(shape=[3,3,128,128],stddev=5e-2))
    #truncated: 정규 분포로서 출력
    b_conv5 = tf.Variable(tf.constant(0.1, shape=[128]))
    h_conv5 = tf.nn.relu(tf.nn.conv2d(h_conv4, W_conv5,strides=[1,1,1,1],padding='SAME')+b_conv5)
    
    # 완전 연결층
    # 2번의 downsampling 이후에, 32 x 32 이미지는 8 x 8 x 128의 Feature map으로 변환
    W_fc1 = tf.Variable(tf.truncated_normal(shape=[8*8*128, 384],stddev=5e-2))
    b_fc1 = tf.Variable(tf.constant(0.1,shape=[384]))
    h_conv5_flat = tf.reshape(h_conv5,[-1,8*8*128])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv5_flat,W_fc1)+b_fc1)
    
    # Dropout - 모델의 복잡도를 컨트롤
    # 특징들의 co-adaptation을 방지
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    # 완전 연결층2
    # 384개의 feature를 10개의 class로 Mapping
    W_fc2 = tf.Variable(tf.truncated_normal(shape=[384, 10],stddev=5e-2))
    b_fc2 = tf.Variable(tf.constant(0.1,shape=[10]))
    logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

- x: Input Data
- y: OutputData
- keep_prob= 드롭아웃에서 드롭하지 않고 유지할 노드 비율

In [12]:
x = tf.placeholder(tf.float32, shape = [None, 32, 32, 3])
y = tf.placeholder(tf.float32, shape = [None, 10])
keep_prob = tf.placeholder(tf.float32)

load_data()함수를 이용하여 CIFAR-10 데이터를 다운로드하고 tf.one_hot API 를 이용해서 스칼라값 형태의 레이블 (0~9)을 One-hot-Encoding 형태로 변환

- squeeze: Removes dimensions of size 1 from the shape of a tensor

In [13]:
(x_train, y_train), (x_test, y_test) = load_data()
#Scalar 현태의 0 ~ 9 형태의 0 ~ 9 을 One - hot Encoding 형태로 변환
y_train_one_hot = tf.squeeze(tf.one_hot(y_train,10),axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test,10),axis=1)

Model 생성
- Loss Function: Cross Entropy
- Optimizer: RMSPropr

In [14]:
y_pred, logits = build_CNN_Classifier(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
train_step = tf.train.RMSPropOptimizer(1e-3).minimize(loss)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


정확도를 계산하는 연산 추가

In [15]:
correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

세션을 열어 그래프를 실행해서 학습을 진행

In [ ]:
with tf.Session() as sess:
    #모든 변수를 초기화
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch = next_batch(128, x_train, y_train_one_hot.eval())
        
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
            loss_print = loss.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
            
            print('반복(Epoch): %d, 정확도: %f, 손실함수: %f'%(i, train_accuracy, loss_print))
            
            #20 % Dropout을 활용하여 학습을 진행
        sess.run(train_step, feed_dict={x: batch[0], y:batch[1], keep_prob:0.8})
            
    #학습이 끝나면 테스트 데이터에 대한 정확도를 출력
    test_accuracy = 0.0
    for i in range(10):
        test_batch = next_batch(1000,x_test,y_test_one_hot.eval())
        test_accuracy = test_accuracy + accuracy.eval(feed_dict={x: test_batch[0], y:test_batch[1], keep_prob: 1.0})
            
    test_accuracy = test_accuracy/10
    print('테스트 데이터 정확도: %f'%(test_accuracy))

반복(Epoch): 0, 정확도: 0.109375, 손실함수: 93.938210
